In [1]:
import cv2
import time
import os
from pathlib import Path
from ultralytics import YOLO

def open_camera(preferred_index=0):
    # Try a few indices in case 0 isn't your webcam
    tried = []
    for idx in [preferred_index, 1, 2, 3]:
        cap = cv2.VideoCapture(idx, cv2.CAP_DSHOW) if cv2.__version__.startswith("4") else cv2.VideoCapture(idx)
        if cap.isOpened():
            return cap, idx
        tried.append(idx)
        if cap:
            cap.release()
    raise RuntimeError(f"Could not open any webcam. Tried indices: {tried}")

def put_text(img, text, org=(10, 30)):
    cv2.putText(img, text, org, cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)

def main():
    # Load YOLOv8 small/fast model (auto-downloads on first use)
    model = YOLO("yolov8n.pt")  # change to 'yolov8s.pt' for more accuracy if your PC is strong

    # Open camera
    cap, cam_idx = open_camera(0)
    # Optional: set a reasonable resolution for speed
    cap.set(cv2.CAP_PROP_FRAME_WIDTH,  1280)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

    print(f"[INFO] Using camera index: {cam_idx}")
    print("[INFO] Press 'q' to quit, 's' to save a snapshot.")

    # Prepare snapshot folder
    out_dir = Path("runs/snapshots")
    out_dir.mkdir(parents=True, exist_ok=True)

    prev_time = time.time()
    fps = 0.0
    frame_count = 0

    try:
        while True:
            ok, frame = cap.read()
            if not ok:
                print("[WARN] Failed to read from camera. Exiting.")
                break

            # Inference (set conf to adjust threshold; imgsz for speed/quality tradeoff)
            results = model.predict(source=frame, conf=0.25, imgsz=640, verbose=False)
            annotated = results[0].plot()  # draw boxes/labels

            # FPS (smoothed)
            now = time.time()
            frame_count += 1
            dt = now - prev_time
            if dt >= 0.5:  # update FPS every 0.5s
                fps = frame_count / dt
                frame_count = 0
                prev_time = now

            put_text(annotated, f"FPS: {fps:.1f}", (10, 30))
            put_text(annotated, "Press 'q' to quit, 's' to save", (10, 60))

            cv2.imshow("YOLOv8 Live Detection", annotated)
            key = cv2.waitKey(1) & 0xFF

            if key == ord('q'):
                break
            elif key == ord('s'):
                ts = time.strftime("%Y%m%d-%H%M%S")
                save_path = out_dir / f"snapshot_{ts}.jpg"
                cv2.imwrite(str(save_path), annotated)
                print(f"[INFO] Saved {save_path}")

    except KeyboardInterrupt:
        pass
    finally:
        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    # Make sure environment variable to disable OpenMP duplicate warning on some systems
    os.environ.setdefault("KMP_DUPLICATE_LIB_OK", "TRUE")
    main()



[INFO] Using camera index: 0
[INFO] Press 'q' to quit, 's' to save a snapshot.


In [3]:
pip install opencv-python


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install ultralytics


   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.1 MB ? eta -:--:--
   ------------------- -------------------- 0.5/1.1 MB 963.5 kB/s eta 0:00:01
   ----------------------------- ---------- 0.8/1.1 MB 969.1 kB/s eta 0:00:01
   ---------------------------------------  1.0/1.1 MB 982.4 kB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 956.0 kB/s eta 0:00:00
   ---------------------------------------- 0.0/38.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/38.5 MB ? eta -:--:--
    --------------------------------------- 0.5/38.5 MB 933.9 kB/s eta 0:00:41
    --------------------------------------- 0.8/38.5 MB 951.7 kB/s eta 0:00:40
    --------------------------------------- 0.8/38.5 MB 951.7 kB/s eta 0:00:40
   - -----------------------------

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\PC\\AppData\\Local\\Temp\\pip-unpack-enn1kvmq\\torch-2.8.0-cp313-cp313-win_amd64.whl'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install ultralytics

  Using cached ultralytics-8.3.185-py3-none-any.whl.metadata (37 kB)
  Using cached torchvision-0.23.0-cp313-cp313-win_amd64.whl.metadata (6.1 kB)
  Using cached ultralytics_thop-2.0.16-py3-none-any.whl.metadata (14 kB)
Using cached ultralytics-8.3.185-py3-none-any.whl (1.1 MB)
Using cached torchvision-0.23.0-cp313-cp313-win_amd64.whl (1.6 MB)
Using cached ultralytics_thop-2.0.16-py3-none-any.whl (28 kB)

   ---------------------------------------- 0/3 [ultralytics-thop]
   ------------- -------------------------- 1/3 [torchvision]
   ------------- -------------------------- 1/3 [torchvision]
   ------------- -------------------------- 1/3 [torchvision]
   ------------- -------------------------- 1/3 [torchvision]
   ------------- -------------------------- 1/3 [torchvision]
   ------------- -------------------------- 1/3 [torchvision]
   ------------- -------------------------- 1/3 [torchvision]
   ------------- -------------------------- 1/3 [torchvision]
   ------------- -----------


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
